# Upload records on the OSSR

To upload records on the OSSR, you may use the [Zenodo client](https://gitlab.in2p3.fr/escape2020/wp3/eossr/-/blob/master/eossr/api/zenodo.py).

The client is also used by the continuous integration to automatise project's uploads from GitLab to Zenodo [check the EOSSR scripts here](https://gitlab.in2p3.fr/escape2020/wp3/eossr/-/tree/master/eossr/scripts).

**However**, to upload records to the OSSR, you **will need** to use your Zenodo token ([create one](https://zenodo.org/account/settings/applications/tokens/new/)).

In [ ]:
import os
token = os.getenv('SANDBOX_ZENODO_TOKEN')  # Replace with your own token

In [ ]:
import json
from eossr.api.zenodo import ZenodoAPI, Record

# Please note that for this demo we are using Zenodo Sandbox.
z = ZenodoAPI(access_token=token, sandbox=True)

## Create a new entry
First we would need to create a new entry

In [ ]:
new_entry = z.create_new_entry()

### Zenodo HTTP status

The function `create_new_entry` would have raised an error in case of failure.    
But if you can its status code meaning using our `ZenodoHTTPStatus` class

In [ ]:
from eossr.api.zenodo.http_status import ZenodoHTTPStatus

In [ ]:
status_new_entry = ZenodoHTTPStatus(new_entry.status_code, new_entry.json())
print(status_new_entry)

## Add files to the entry

Then we upload to the new entry a file (let's say a `codemeta.json` metadata file)

In [ ]:
deposit_id = new_entry.json()['id']
upload_file = z.upload_file_entry(deposit_id, 
                                  name_file='codemeta.json', 
                                  path_file='../../codemeta.json')

In [ ]:
status_upload = ZenodoHTTPStatus(upload_file.status_code, upload_file.json())
print(status_upload)

## Add metadata to the entry

We add metadata to the Zenodo entry using the EOSSR codemeta2zenodo conversor

In [ ]:
from eossr.metadata.codemeta2zenodo import parse_codemeta_and_write_zenodo_metadata_file

In [ ]:
parse_codemeta_and_write_zenodo_metadata_file(codemeta_filename='../../codemeta.json', 
                                              outdir='.',
                                              add_escape2020=True,
                                              overwrite=True,
                                             )

In [ ]:
# Let's have a look to the output to the .zenodo.json file
! cat .zenodo.json

In [ ]:
with open('.zenodo.json') as f:
    zenodo_metadata = json.load(f)
update_metadata_info = z.set_deposit_metadata(deposit_id, zenodo_metadata)

In [ ]:
status_metadata = ZenodoHTTPStatus(update_metadata_info.status_code, update_metadata_info.json())
print(status_metadata)

### Check the content of the new entry

The `request` answers contain all the information of the current upload.    
The json dictionnary can be treated as a standard Zenodo Record:

In [ ]:
new_record = Record(update_metadata_info.json())
new_record

In [ ]:
new_record.print_info()

We can check the files that we have already uploaded

In [ ]:
new_record.data['files']

Or the metadata updated 

In [ ]:
new_record.data['metadata']

## Publish the entry

Finally we can publish the new entry. 
However in this case, because this is just a demo, we will just erase the entry.

In [ ]:
# In case you would like to publish the entry, just uncomment the following lines.

# # publish = z.publish_entry(deposit_id)
# # status = ZenodoHTTPStatus(publish.status_code, publish.json())

In [ ]:
from eossr.api.zenodo.http_status import HTTPStatusError

In [ ]:
erase_entry = z.erase_entry(deposit_id)